### 1. Get data:

In [2]:
import numpy as np
from astropy.stats import LombScargle
import matplotlib.pyplot as plt
%matplotlib inline
# Read the data
date, rv, rverr, ha, haerr, nad, naderr, fwhm, fwhmerr, bis, biserr = \
    np.loadtxt("../data/dbf_HARPS.txt", unpack=True, \
    usecols=[1,2,3,4,5,6,7,8,9,10,11], skiprows=21)
dateH, rvH, rverrH, haH, haerrH = \
    np.loadtxt("../data/dbf_HIRES.txt", unpack=True, \
    usecols=[1,2,3,4,5])
first = date[0]
date -= first
dateH -= first
# Put together the HIRES and HARPS measurements of RV and Halpha
alldates = np.concatenate((date, dateH))
allRV = np.concatenate((rv, rvH))
allRVerr = np.concatenate((rverr, rverrH))
allha = np.concatenate((ha, haH))
allhaerr = np.concatenate((haerr, haerrH))
sub = np.argsort(alldates)
alldates = alldates[sub]
allRV = allRV[sub]
allRVerr = allRVerr[sub]
allha = allha[sub]
allhaerr = allhaerr[sub]
# Compute the periodograms
xiR, powR = LombScargle(alldates, allRV, allRVerr).autopower()
xiH, powH = LombScargle(alldates, allha, allhaerr).autopower()

### 2. Gaussian processes:

In [3]:
# To install george: $ conda install -c conda-forge george
# Most of this code is borrowed from a tutorial in the george documentation:
# https://george.readthedocs.io/en/latest/tutorials/hyper/
from george import kernels
from george import GP
from george import modeling
from scipy.optimize import minimize
from scipy.optimize import Bounds

In [ ]:
# Initial guesses for the hyperparameters
constant_guess = 8. # Amplitude of squared-exponential decorrelation kernel
metric_guess = 50000. # Decorrelation timescale SQUARED
period_guess = 110 # Best-fit rotation period from H alpha
gamma_guess = 3. # Represents (1 / Amplitude) of periodic RV signal

# Create the kernels
kexp2 = constant_guess * kernels.ExpSquaredKernel(metric=metric_guess)
kper = kernels.ExpSine2Kernel(gamma=gamma_guess, log_period=np.log(period_guess))
krot = kexp2 * kper


def whitenoise_RV(time):
    return np.log(allRVerr**2)

# Initialize the Gaussian process. 
gp = GP(krot, fit_kernel=True, mean=np.mean(allRV), \
        white_noise=modeling.CallableModel(whitenoise, gradient=None))